In [1]:
!git clone https://github.com/pasapas321/bench.git

Cloning into 'bench'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 193 (delta 70), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (193/193), 331.86 KiB | 4.42 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [2]:
import sys

sys.path.append('/kaggle/working/bench')

In [3]:
!pip install -U "huggingface_hub[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 9.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0


In [6]:
!pip install transformers==4.48.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 61.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0


In [7]:
!pip install shortuuid

In [8]:
!pip install fschat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.4/744.4 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 5.3 MB/s eta 0:00:00
  Created wheel for wavedrom: filename=wavedrom-2.0.3.post3-py2.py3-none-any.whl size=30052 sha256=c134b12fd6bac8591dd8808342bdee696e795b15f0e67c72d2c951f15d02dc62
  Stored in directory: /root/.cache/pip/wheels/9c/52/8c/38b454b42f712f325e26f633287

In [9]:
!pip install openai

In [10]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 5.9 MB/s eta 0:00:00a 0:00:01


In [11]:
!pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 8.8 MB/s eta 0:00:00ta 0:00:01


In [11]:
!pip install auto_gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 61.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 84.9 MB/s eta 0:00:00:00:010:01


In [12]:
# !pip install shortuuid
# !pip install fschat
# !pip install openai
# !pip install anthropic
# !pip install optimum
# !pip install auto_gptq

In [15]:
"""Generate answers with local models.

Usage:
python3 gen_model_answer.py --model-path lmsys/fastchat-t5-3b-v1.0 --model-id fastchat-t5-3b-v1.0
"""
import argparse

from evaluation.eval import run_eval, reorg_answer_file

from model.specexec.spec_generator import create_spec_generator
from model.specexec.specdec import utils
from model.specexec.specdec.trees import Tree

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np

device = torch.device("cuda:0")

def specexec_forward(inputs, model, tokenizer, max_new_tokens, spec_generator=None,
                     seed=0, max_budget=16, max_beam_len=32, device=torch.device("cuda:0")):
    input_ids = inputs.input_ids
    assert input_ids.shape[0] == 1, "Only support batch size 1 for now!!"
    input_ids = input_ids.clone()
    accept_length_list = []
    
    spec_generator.prefix_tokens = input_ids[0]
    spec_generator.original_num_tokens = len(spec_generator.prefix_tokens)

    utils.set_seed(seed)
    torch.cuda.reset_peak_memory_stats()

    if hasattr(spec_generator, "tree"):
        del spec_generator.tree
    spec_generator.reset_engines(prefix_len=len(spec_generator.prefix_tokens), max_budget=max_budget, max_new_tokens=max_new_tokens)

    spec_generator.tree = Tree(prefix_tokens=spec_generator.prefix_tokens, draft_engine=spec_generator.draft_engine, tokenizer=spec_generator.tokenizer)
    spec_generator.levels = spec_generator.get_tree_levels()  # in case the child class works with fixed trees

    # main generation cycle
    iter = 1
    eos_flag = False

    while len(spec_generator.prefix_tokens) < max_new_tokens + spec_generator.original_num_tokens and not eos_flag:
        stats0 = spec_generator.grow_tree(prefix_tokens=spec_generator.prefix_tokens, max_budget=max_budget, max_beam_len=max_beam_len)
        stats1, fresh_tokens = spec_generator.validate_tree()
        torch.cuda.empty_cache()

        if spec_generator.tokenizer.eos_token_id in fresh_tokens:
            fresh_tokens = fresh_tokens[: fresh_tokens.index(spec_generator.tokenizer.eos_token_id)]
            eos_flag = True

        if len(fresh_tokens) > 0:
            fresh_tokens_tensor = torch.tensor(fresh_tokens).to(device)
            spec_generator.prefix_tokens = torch.cat((spec_generator.prefix_tokens, fresh_tokens_tensor))

        iter += 1
        accept_length_list.append(len(fresh_tokens))
    
    new_token = len(spec_generator.prefix_tokens) - spec_generator.original_num_tokens
    output_ids = spec_generator.prefix_tokens.unsqueeze(0)

    return output_ids, new_token, iter, accept_length_list
    
question_file = f"/kaggle/working/bench/data/spec_bench/question.jsonl"
answer_file = f"/kaggle/working/bench/data/spec_bench/model_answer/specexec_model.jsonl"

print(f"Output to {answer_file}")

model_name_0 = 'JackFram/llama-68m'
model_name_1 = с

spec_generator = create_spec_generator(
    model_name_0=model_name_0,
    model_name_1=model_name_1,
    draft_engine_class='padded',
    gen_type='sx_base',
    offload=False,
    tree_max_len=4096
)

#tokenizer = AutoTokenizer.from_pretrained(model_name_0, legacy=False)
tokenizer = spec_generator.tokenizer

model = AutoModelForCausalLM.from_pretrained(model_name_1, device_map="auto")

run_eval(
    model=model,
    tokenizer=tokenizer,
    forward_func=specexec_forward,
    model_id='specexec_model',
    question_file=question_file,
    question_begin=0,
    question_end=480,
    answer_file=answer_file,
    max_new_tokens=32,
    num_choices=1,
    num_gpus_per_model=1,
    num_gpus_total=1,
    spec_generator=spec_generator,
    seed=0,
    max_budget=16,
    max_beam_len=32,
    device=device
)

reorg_answer_file(answer_file)

Output to /kaggle/working/bench/data/spec_bench/model_answer/specexec_model.jsonl
Check model training state: False
CUDA VISIBLE DEVICES: None
Warmup done


100%|██████████| 480/480 [36:23<00:00,  4.55s/it]

#Mean accepted tokens:  1.9532740327939129
